In [ ]:
%pip install pandas
%pip install numpy
%pip install sklearn
%pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('data/out.csv')

data = data.drop('Year', axis=1)
data = data.drop('Country', axis=1)

In [ ]:
train_size = int(0.3 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

In [ ]:
svm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
svm.fit(train_data)

In [ ]:
val_predictions = svm.predict(val_data)
val_labels = np.ones(len(val_data))
val_labels[val_predictions == -1] = 0  # Anomalies are labeled as -1

if np.sum(val_labels == 1) == 0 or np.sum(val_predictions == 1) == 0:
    # If there are no true positives or false positives, set precision and recall to 0
    precision = 0
    recall = 0
    f1 = 0
else:
    precision = precision_score(val_labels, val_predictions, average='weighted')
    recall = recall_score(val_labels, val_predictions, average='weighted')
    f1 = f1_score(val_labels, val_predictions, average='weighted')


print(f'Precision: {precision:.3f}, Recall: {recall:.3f}, F1-score: {f1:.3f}')

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(data.index, data['Homicide Rate'], label='Normal', s=10)
anomaly_mask = (val_predictions == -1)
anomaly_data = val_data.loc[anomaly_mask]
plt.scatter(anomaly_data.index, anomaly_data['Homicide Rate'], label='Anomaly', s=50, color='red')
plt.title('One-class SVM Anomaly Detection')
plt.xlabel('Sample Index')
plt.ylabel('Homicide Rate')
plt.legend()
plt.show()